# 👁️ Diabetic Retinopathy Detection using Agentic AI & Deep Learning

This notebook trains a deep-learning model to classify:
- **DR**   → Diabetic Retinopathy  
- **No_DR** → Healthy retina

Dataset used:
Diagnosis of Diabetic Retinopathy  
https://www.kaggle.com/datasets/pkdarabi/diagnosis-of-diabetic-retinopathy

The notebook includes:

✔ Dataset loading from Kaggle  
✔ Train/Validation/Test directory structure  
✔ Image augmentation  
✔ CNN Model (custom)  
✔ Performance evaluation + confusion matrix  
✔ Model saving as `.h5` for deployment  
✔ Instructions to download/export trained models  

This is the **Kaggle training notebook**.  
Later, you will integrate this trained model into your **Colab Web App**.


## 📂 Dataset Usage Instructions (Kaggle)

This dataset contains three folders:

- train/
- valid/
- test/

Each folder contains two classes:
- DR     → Eyes with Diabetic Retinopathy  
- No_DR  → Healthy retina images

After adding the dataset to your Kaggle Notebook:

1. Click **Add Data → Search "Diagnosis of Diabetic Retinopathy"**
2. Attach the dataset.
3. Kaggle automatically mounts it under:
   /kaggle/input/diagnosis-of-diabetic-retinopathy/

Inside this folder you will find:

Diagnosis of Diabetic Retinopathy/
 ├── train/
 ├── valid/
 ├── test/

You must set your `base_dir` accordingly:

base_dir = '/kaggle/input/diagnosis-of-diabetic-retinopathy/Diagnosis of Diabetic Retinopathy'


## 🖼️ Image Preprocessing & Augmentation

We resize all images to **224 × 224** and normalize pixel values to **[0, 1]**.

Training data uses augmentation:
- rotation
- zooming
- shifting
- brightness changes
- horizontal flips

Validation & Test sets use **only rescaling**, no augmentation.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import itertools

# Paths — adjust as per your folder structure
base_dir = '/kaggle/input/diagnosis-of-diabetic-retinopathy/Diagnosis of Diabetic Retinopathy'   # change this to your root folder
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'valid')
test_dir  = os.path.join(base_dir, 'test')

# Image parameters
IMG_HEIGHT = 224
IMG_WIDTH  = 224
BATCH_SIZE = 32
NUM_CLASSES = 2    # DR vs No_DR

# Data generators with augmentation for training
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.15,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen= ImageDataGenerator(rescale=1./255)

# ## 🧠 CNN Model Architecture

# We build a custom Convolutional Neural Network with:

# - 3 Convolutional blocks (32 → 64 → 128 filters)
# - Batch Normalization after each conv layer
# - Max Pooling to reduce spatial size
# - Dense layer with 128 neurons
# - Dropout (0.5) to prevent overfitting
# - Sigmoid output (binary classification)

# Optimizer: Adam
# Loss: Binary Crossentropy
# Metrics: Accuracy


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'     # since two classes
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False    # for evaluation, keep ordering
)

print("Classes:", train_generator.class_indices)


## 🧠 CNN Model Architecture

We build a custom Convolutional Neural Network with:

- 3 Convolutional blocks (32 → 64 → 128 filters)
- Batch Normalization after each conv layer
- Max Pooling to reduce spatial size
- Dense layer with 128 neurons
- Dropout (0.5) to prevent overfitting
- Sigmoid output (binary classification)

Optimizer: Adam  
Loss: Binary Crossentropy  
Metrics: Accuracy  


In [ ]:
from tensorflow.keras import layers, models

model_cnn = keras.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model_cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_cnn.summary()

## 📉 Learning Rate Scheduler

We use ReduceLROnPlateau to automatically lower the LR when validation loss stagnates:

- factor = 0.2  
- patience = 2  
- min_lr = 1e-6  

This helps stabilize training and improve convergence.


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

## 🚀 Model Training

We train for **20 epochs** (can be increased for better accuracy).  
Both training & validation accuracy will be plotted at the end if needed.

Use GPU runtime on Kaggle for much faster training.


In [ ]:
history_cnn = model_cnn.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,              # increase epochs
    callbacks=[reduce_lr]
)


## 📊 Model Evaluation (Test Set)

We evaluate the model on the test set to measure:

- Precision
- Recall
- F1 Score
- Overall Accuracy

We also compute a **confusion matrix** to visualize correct vs incorrect predictions.


In [ ]:
import seaborn as sns
# Get predictions
test_steps = test_generator.samples // BATCH_SIZE + 1
pred_probs = model_cnn.predict(test_generator, steps=test_steps)
pred_labels = (pred_probs > 0.5).astype(int).reshape(-1)

true_labels = test_generator.classes  # since shuffle=False
class_names = list(test_generator.class_indices.keys())

# Print classification report
print(classification_report(true_labels, pred_labels, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

sns.heatmap(cm, annot=True, fmt='d', cmap='Purples')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("CNN Confusion Matrix")
plt.show()


## 💾 Saving the Trained Model (for Colab Web App)

The final model is saved in .h5 format:

model_cnn.save('cnn_advanced_model.h5')

This file will appear in:
- /kaggle/working/cnn_advanced_model.h5

### 📥 Downloading from Kaggle:

Go to the right sidebar → "Output Files" → download cnn_advanced_model.h5

### 📌 This model will be used later in the **Colab Web App**.


In [ ]:
from tensorflow.keras.models import load_model

# Save as HDF5
model_cnn.save('cnn_advanced_model.h5')
print("CNN model saved as cnn_advanced_model.h5")

# Load later
loaded_cnn_h5 = load_model('cnn_advanced_model.h5')

# Run this in Colab

# 👁️ Diabetic Retinopathy Detection Web App (Agentic AI + Deep Learning)

This notebook deploys a full AI-powered web application using:

✔ Pre-trained CNN model from Kaggle (DR vs No_DR)  
✔ Severity classification based on CNN feature extraction  
✔ Mistral-7B-Instruct LLM (4-bit, GPU optimized)  
✔ Personalized recovery steps using Agentic AI  
✔ Flask + Ngrok for live public hosting  

To run successfully:

1. Upload your trained model file: **cnn_advanced_model.h5**
2. Authenticate with Hugging Face (LLM access)
3. Mount Google Drive
4. Start Flask + Ngrok to launch the app


## 📦 Install All Required Dependencies

This installs:

- Flask (backend)
- PyNgrok (public URL for your app)
- TensorFlow (loads the trained CNN model)
- Transformers + BitsAndBytes (loads Mistral-7B 4-bit)
- SentencePiece (required for tokenizer)

These libraries enable:
- Model loading
- Image preprocessing
- LLM generation
- Web UI execution


In [ ]:
!pip install -q flask pyngrok tensorflow pillow requests

In [ ]:
!pip install transformers accelerate bitsandbytes sentencepiece


## 📂 Connect Google Drive

Your trained CNN model (cnn_advanced_model.h5) must be stored in Drive.

Steps:
1. Upload the model into any folder inside MyDrive
2. Update the model path in the code if needed
3. Verify the model using `!ls` command


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Verify model file path
!ls "/content/drive/MyDrive/Colab Notebooks/Varma sir projects/Diabetic Retinopathy/cnn_advanced_model.h5"

## 🔑 Hugging Face Token Setup (Required)

The LLM (Mistral-7B) is hosted on HuggingFace.

To use it:
1. Go to: https://huggingface.co/settings/tokens
2. Click "New Token"
3. Select: Permission = Read
4. Copy the token
5. Paste token into: login(token="YOUR_TOKEN")

⚠️ Never share your token publicly.


In [ ]:
from huggingface_hub import login
login(token="YOUR_TOKEN")   # Replace with your HF token

## 📁 Create App Structure (Templates, Static, Uploads)

Flask requires project folders:
- templates → HTML files
- static → CSS files
- uploads → where user images go

These directories will be auto-created.


In [ ]:
!mkdir -p templates static uploads


## 🔐 Store HF Token in Environment Variable

To allow the LLM to authenticate internally,
we export the HuggingFace token as HF_TOKEN.

This avoids hardcoding the token inside model loading code.


In [ ]:
import os
os.environ["HF_TOKEN"] = "your token"


# 🧠 App Backend Overview (app.py)

The backend performs:

1. Load pretrained CNN model from Drive
2. Build feature extractor for advanced severity analysis
3. Preprocess uploaded images
4. Predict:
   - DR vs No_DR
   - Confidence score
5. Extract numerical retina feature patterns
6. Severity classification (Early → Severe)
7. Use LLM (Mistral-7B) to generate personalized recovery plans
8. Return full JSON response to frontend

⚠️ Note:
Some layers in CNN may differ. If feature extraction fails,
fallback measures are automatically used.


In [ ]:
%%writefile app.py
import os
import tensorflow as tf
from flask import Flask, request, render_template, jsonify
from PIL import Image
import numpy as np
import torch
import re
from tensorflow.keras.models import Model

# ## 🧠 Load Trained DR Classification Model

# We load the model exported from Kaggle:
# cnn_advanced_model.h5

# The model path:
# "/content/drive/MyDrive/.../cnn_advanced_model.h5"

# Make sure:
# ✔ The path is correct
# ✔ The file name matches
# ✔ Model is inside Google Drive


# ----------------------------------------------------------
# LOAD RETINOPATHY CNN MODEL
# ----------------------------------------------------------
model_path = "/content/drive/MyDrive/Colab Notebooks/Varma sir projects/Diabetic Retinopathy/cnn_advanced_model.h5"
base_model = tf.keras.models.load_model(model_path)

# Build the model by passing a dummy input to establish the computational graph
dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
_ = base_model.predict(dummy_input, verbose=0)

# Now create feature extractor from intermediate layer
try:
    # Try to get the layer before the final dense layers
    feature_layer = base_model.layers[-4]  # Adjust if needed
    feature_extractor = Model(
        inputs=base_model.input,
        outputs=[feature_layer.output, base_model.output]
    )
except Exception as e:
    print(f"Warning: Could not create feature extractor: {e}")
    print("Using base model only for predictions")
    feature_extractor = None

app = Flask(__name__)

# ----------------------------------------------------------
# IMAGE PREPROCESSOR
# ----------------------------------------------------------
def preprocess_image(image):
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    return np.expand_dims(image, axis=0)


# ## 🔬 AI Retina Feature Extraction

# We extract additional signals beyond the basic prediction:
# - activation strength
# - pattern complexity
# - max feature response
# - affected_area_ratio

# These features help determine:
# ✔ severity
# ✔ depth of abnormalities
# ✔ AI confidence reasoning



# ----------------------------------------------------------
# EXTRACT VISUAL FEATURES FOR CONTEXT
# ----------------------------------------------------------
def analyze_image_features(input_arr):
    """
    Extract features from the CNN to understand what patterns the model detected.
    This gives us context about the severity and characteristics of DR.
    """
    if feature_extractor is None:
        # Fallback: use simple variance analysis on the input image
        img_variance = np.var(input_arr)
        img_mean = np.mean(input_arr)
        return {
            'activation_strength': float(img_mean),
            'pattern_complexity': float(img_variance),
            'max_response': 1.0,
            'affected_area_ratio': 0.5
        }

    try:
        features, prediction = feature_extractor.predict(input_arr, verbose=0)

        # Analyze feature activation patterns
        feature_mean = np.mean(features)
        feature_std = np.std(features)
        feature_max = np.max(features)
        high_activation_ratio = np.sum(features > 0.5) / features.size

        return {
            'activation_strength': float(feature_mean),
            'pattern_complexity': float(feature_std),
            'max_response': float(feature_max),
            'affected_area_ratio': float(high_activation_ratio)
        }
    except Exception as e:
        print(f"Error in feature extraction: {e}")
        # Fallback values
        return {
            'activation_strength': 0.4,
            'pattern_complexity': 0.3,
            'max_response': 0.8,
            'affected_area_ratio': 0.45
        }


# ## 🤖 Load Mistral-7B-Instruct (4-bit)

# The LLM is used to:
# - interpret AI findings
# - generate personalized recovery steps
# - produce structured recommendations

# We use BitsAndBytesConfig to load the model in 4-bit for speed + memory optimization.

# Requirements:
# ✔ GPU runtime
# ✔ HF token set
# ✔ Internet on first model load


# ----------------------------------------------------------
# LOAD LLM (Mistral-7B-Instruct, 4-bit, GPU)
# ----------------------------------------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print("Loading LLM tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
print("Loading LLM model (this may take a minute)...")
llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb,
    device_map="auto",
    torch_dtype=torch.float16
)
print("LLM loaded successfully!")


# ## ⚠️ Severity Classification Engine

# Based on:
# - CNN confidence
# - Activation intensity
# - Pattern complexity
# - Affected retinal area

# We classify DR as:
# - Early/Mild
# - Moderate
# - Moderately Severe
# - Severe/Advanced



# ----------------------------------------------------------
# ENHANCED SEVERITY CLASSIFICATION
# ----------------------------------------------------------
def classify_severity(confidence, features):
    """
    Multi-dimensional severity assessment using both prediction confidence
    and extracted visual features from the CNN.
    """
    activation = features['activation_strength']
    complexity = features['pattern_complexity']
    affected_ratio = features['affected_area_ratio']

    # Weighted severity score
    severity_score = (
        confidence * 0.4 +
        min(activation * 2, 1.0) * 0.25 +  # Normalized activation
        min(complexity * 3, 1.0) * 0.15 +   # Normalized complexity
        affected_ratio * 0.2
    )

    if severity_score < 0.65:
        return "Early/Mild", "mild changes detected"
    elif severity_score < 0.80:
        return "Moderate", "moderate retinal changes observed"
    elif severity_score < 0.92:
        return "Moderately Severe", "significant retinal abnormalities detected"
    else:
        return "Severe/Advanced", "advanced retinal damage patterns identified"


# ----------------------------------------------------------
# POST-PROCESS HELPERS
# ----------------------------------------------------------
def extract_bullets(text, max_bullets=10):
    """Extract clean bullet points from LLM output"""
    t = text.strip()

    # Remove instruction artifacts
    t = re.sub(r"^\[?/?INST\]?[:\s]*", "", t, flags=re.IGNORECASE).strip()
    t = re.sub(r"^(response|answer|here are|steps)\s*:?", "", t, flags=re.IGNORECASE).strip()

    lines = [ln.strip() for ln in t.splitlines() if ln.strip()]
    bullets = []

    for ln in lines:
        if re.match(r"^(\d+[\.\)]\s+|-|•|\*)", ln):
            clean = re.sub(r"^(\d+[\.\)]\s+|-|•|\*)\s*", "", ln).strip()
            if len(clean) > 15:  # Skip very short items
                bullets.append(clean)
        elif len(ln) > 20 and not re.search(r"(instruction|requirement|context)", ln.lower()):
            bullets.append(ln)

        if len(bullets) >= max_bullets:
            break

    # If not enough bullets, split longer items
    if len(bullets) < 6:
        expanded = []
        for item in bullets:
            if ';' in item or ', and ' in item:
                parts = re.split(r';\s*|,\s*and\s+', item)
                expanded.extend([p.strip() for p in parts if len(p.strip()) > 15])
            else:
                expanded.append(item)
        bullets = expanded[:max_bullets]

    # Deduplicate
    seen = set()
    final = []
    for b in bullets:
        b_lower = b.lower()
        if b_lower not in seen and len(b) > 15:
            final.append(b)
            seen.add(b_lower)

    return final[:max_bullets]

# # 🩺 Personalized Recovery Plan (Agentic AI)

# The LLM uses image-derived biomarkers + severity label to generate:

# ✔ Medical steps
# ✔ Diagnostic tests required
# ✔ Lifestyle recommendations
# ✔ Warning signs
# ✔ Follow-up frequency
# ✔ Evidence-based treatment suggestions

# This section produces a highly personalized explanation for each patient.


# ----------------------------------------------------------
# AGENTIC AI GENERATION WITH IMAGE CONTEXT
# ----------------------------------------------------------
def generate_recovery_steps(confidence, features, severity_label, severity_desc):
    """
    Generate personalized recovery steps using actual image analysis context.
    Each image will produce unique recommendations based on detected patterns.
    """

    # Create detailed context from feature analysis
    activation = features['activation_strength']
    complexity = features['pattern_complexity']
    affected_ratio = features['affected_area_ratio'] * 100

    # Determine focus areas based on detected patterns
    focus_areas = []
    if affected_ratio > 40:
        focus_areas.append("widespread retinal involvement")
    if complexity > 0.3:
        focus_areas.append("complex vascular changes")
    if activation > 0.4:
        focus_areas.append("significant neural activation patterns")

    focus_text = ", ".join(focus_areas) if focus_areas else "early retinal changes"

    user_prompt = f"""
You are an ophthalmology AI assistant. Generate specific, actionable recovery and management steps for a diabetic retinopathy patient.

DETECTION DETAILS:
- Severity Classification: {severity_label}
- Pattern Analysis: {severity_desc}
- AI Confidence: {confidence:.1%}
- Affected Retinal Area: ~{affected_ratio:.1f}%
- Detected Patterns: {focus_text}

GENERATE 8-10 PERSONALIZED RECOMMENDATIONS:
Focus on these categories based on the severity:
1. Immediate medical actions (specific to {severity_label} cases)
2. Essential diagnostic tests needed for this severity level
3. Blood sugar management strategies
4. Eye-specific care routines
5. Lifestyle modifications relevant to detected patterns
6. Warning signs to monitor
7. Follow-up schedule appropriate for {severity_label} DR
8. Treatment options typically used for this stage

REQUIREMENTS:
- Be specific and actionable (avoid generic advice)
- Tailor recommendations to {severity_label} severity
- Use clear, supportive language
- Include specific test names (Fundus photography, OCT, Fluorescein angiography)
- Mention realistic timeframes for follow-ups
- Address both medical and lifestyle factors
- End with: "This is AI-assisted analysis, not a diagnosis. Consult an ophthalmologist immediately."

Provide ONLY the numbered list of recommendations, nothing else.
"""

    final_prompt = f"<s>[INST] {user_prompt.strip()} [/INST]"

    try:
        inputs = tokenizer(final_prompt, return_tensors="pt").to(llm_model.device)
        input_len = inputs["input_ids"].shape[1]

        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=280,
            temperature=0.7,  # Increased for more varied outputs
            top_p=0.92,
            top_k=50,
            do_sample=True,
            repetition_penalty=1.2,  # Reduce repetition
            eos_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id is not None else 2,
        )

        out_ids = outputs[0]
        gen_ids = out_ids[input_len:] if out_ids.shape[0] > input_len else out_ids
        gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

        bullets = extract_bullets(gen_text, max_bullets=10)

        if not bullets or len(bullets) < 3:
            # Fallback with contextual steps
            bullets = generate_fallback_steps(severity_label, affected_ratio)

        formatted = "\n".join([f"{i+1}. {b}" for i, b in enumerate(bullets)])

        if "not a diagnosis" not in formatted.lower() and "ai-assisted" not in formatted.lower():
            formatted += "\n\n⚠️ This is AI-assisted analysis, not a diagnosis. Consult an ophthalmologist immediately."

        return formatted

    except Exception as e:
        print(f"LLM generation error: {e}")
        return generate_fallback_steps(severity_label, affected_ratio)


def generate_fallback_steps(severity, affected_ratio):
    """Fallback steps if LLM generation fails"""
    steps = [
        f"Schedule an urgent appointment with a retinal specialist for {severity.lower()} diabetic retinopathy evaluation",
        "Get a comprehensive dilated fundus examination within the next 1-2 weeks",
        f"Request Optical Coherence Tomography (OCT) scan to assess macular involvement (approximately {affected_ratio:.0f}% area affected)",
        "Monitor blood glucose levels 4 times daily and maintain HbA1c below 7.0%",
        "Check blood pressure twice daily; target <130/80 mmHg to reduce progression risk",
        "Begin strict dietary control: limit refined carbs, increase omega-3 rich foods",
        "Implement daily moderate exercise (30-45 mins walking) after physician clearance",
        "Watch for warning signs: sudden vision loss, floaters, flashing lights, dark spots",
        f"For {severity.lower()} cases, expect follow-up every 3-4 months or as advised",
        "Discuss laser photocoagulation or anti-VEGF therapy options with your ophthalmologist"
    ]

    formatted = "\n".join([f"{i+1}. {s}" for i, s in enumerate(steps)])
    formatted += "\n\n⚠️ This is AI-assisted analysis, not a diagnosis. Consult an ophthalmologist immediately."
    return formatted

# ## 🌐 Flask Routes

# Endpoints:

# / → Welcome page
# /upload → Upload image UI
# /predict → API endpoint (POST image)

# The predict route returns:
# - Prediction label
# - Confidence
# - Severity
# - Affected area
# - Recovery steps (AI generated)
# - Disclaimer


# ----------------------------------------------------------
# ROUTES
# ----------------------------------------------------------
@app.route("/")
def welcome():
    return render_template("welcome.html")


@app.route("/upload")
def upload():
    return render_template("index.html")


@app.route("/predict", methods=["POST"])
def predict():
    if "image" not in request.files:
        return jsonify({"error": "No image uploaded"}), 400

    try:
        file = request.files["image"]
        image = Image.open(file).convert("RGB")
        input_arr = preprocess_image(image)

        # Get prediction
        pred = float(base_model.predict(input_arr, verbose=0)[0][0])

        label = "No_DR" if pred > 0.5 else "DR"
        confidence = pred if label == "No_DR" else (1 - pred)

        response = {
            "label_raw": label,
            "confidence": round(confidence, 4)
        }

        if label == "No_DR":
            response["prediction"] = "👁️✨ Retina appears healthy with no signs of diabetic retinopathy detected."
            response["message"] = "Continue regular eye checkups and maintain good glucose control."
        else:
            # Extract detailed features for DR cases
            features = analyze_image_features(input_arr)
            severity_label, severity_desc = classify_severity(confidence, features)

            response["prediction"] = f"⚠️ Diabetic Retinopathy Detected: {severity_label} stage"
            response["severity"] = severity_label
            response["severity_description"] = severity_desc.capitalize()
            response["affected_area"] = f"{features['affected_area_ratio']*100:.1f}%"

            # Generate personalized steps
            steps = generate_recovery_steps(confidence, features, severity_label, severity_desc)
            response["recovery_steps"] = steps
            response["disclaimer"] = "⚠️ This is AI-assisted analysis. Immediate consultation with a licensed ophthalmologist is strongly recommended."

        return jsonify(response)

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        print(f"Error in predict route: {error_details}")
        return jsonify({"error": f"Processing error: {str(e)}"}), 500


# ----------------------------------------------------------
# RUN FLASK
# ----------------------------------------------------------
if __name__ == "__main__":
    print("=" * 60)
    print("🚀 Flask app starting...")
    print("=" * 60)
    app.run(host="0.0.0.0", port=8000, debug=True)

## 🎨 Frontend Templates

HTML templates include:

1. welcome.html → Landing page  
2. index.html → Image upload + analysis interface  
3. style.css → Visual styling  

These files work automatically with Flask.


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>DR Detection - Upload</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>

<body>
  <div class="header">
    <h1>👁️ Diabetic Retinopathy Detection</h1>
    <p class="tagline">AI-Powered Retinal Analysis</p>
  </div>

  <div class="container">
    <div class="upload-section">
      <h2>Upload Retinal Image</h2>
      <p class="instruction">Select a fundus photograph for analysis</p>

      <form id="uploadForm" onsubmit="uploadImage(event)">
        <div class="file-input-wrapper">
          <input type="file" id="image" name="image" accept="image/*" required onchange="previewImage(event)" />
          <label for="image" class="file-label">
            <span id="fileName">Choose Image File</span>
            <span class="upload-icon">📁</span>
          </label>
        </div>

        <div id="imagePreview" class="image-preview"></div>

        <button type="submit" class="submit-btn" id="submitBtn">
          <span class="btn-text">🔍 Analyze Image</span>
        </button>
      </form>
    </div>

    <div id="resultsSection" class="results-section hidden">
      <div class="result-header">
        <h2>Analysis Results</h2>
      </div>

      <div id="predictionResult" class="result-content"></div>
    </div>
  </div>

  <script>
    function previewImage(event) {
      const file = event.target.files[0];
      const preview = document.getElementById('imagePreview');
      const fileName = document.getElementById('fileName');

      if (file) {
        fileName.textContent = file.name;
        const reader = new FileReader();
        reader.onload = function(e) {
          preview.innerHTML = `<img src="${e.target.result}" alt="Preview" />`;
          preview.style.display = 'block';
        };
        reader.readAsDataURL(file);
      }
    }

    function uploadImage(event) {
      event.preventDefault();

      const file = document.getElementById("image").files[0];
      if (!file) {
        alert("Please select an image file");
        return;
      }

      const fd = new FormData();
      fd.append("image", file);

      const resultsSection = document.getElementById("resultsSection");
      const resultDiv = document.getElementById("predictionResult");
      const submitBtn = document.getElementById("submitBtn");

      // Show loading state
      submitBtn.disabled = true;
      submitBtn.innerHTML = '<span class="btn-text">🔄 Analyzing...</span>';
      resultsSection.classList.remove('hidden');
      resultDiv.innerHTML = `
        <div class="loading">
          <div class="spinner"></div>
          <p>Processing retinal image...</p>
          <p class="loading-sub">Running AI analysis</p>
        </div>
      `;

      fetch("/predict", { method: "POST", body: fd })
        .then(res => res.json())
        .then(data => {
          if (data.error) {
            resultDiv.innerHTML = `<div class="error">❌ Error: ${data.error}</div>`;
            return;
          }

          let html = `<div class="result-card">`;

          // Main prediction
          html += `<div class="prediction-main">
            <h3>Diagnosis</h3>
            <p class="prediction-text">${data.prediction}</p>
            <div class="confidence-bar">
              <div class="confidence-label">AI Confidence</div>
              <div class="progress-bar">
                <div class="progress-fill" style="width: ${(data.confidence * 100).toFixed(1)}%"></div>
              </div>
              <div class="confidence-value">${(data.confidence * 100).toFixed(1)}%</div>
            </div>
          </div>`;

          // If DR detected, show detailed analysis
          if (data.recovery_steps) {
            html += `<div class="severity-info">
              <div class="severity-badge severity-${data.severity.toLowerCase().replace(/\s+/g, '-')}">
                ${data.severity}
              </div>
              <p class="severity-desc">${data.severity_description}</p>
              <p class="affected-area">Affected Area: <strong>${data.affected_area}</strong></p>
            </div>`;

            html += `<div class="recovery-section">
              <h3>📋 Recommended Care Steps</h3>
              <div class="steps-container">
                <pre class="steps-text">${data.recovery_steps}</pre>
              </div>
            </div>`;

            html += `<div class="disclaimer">
              ${data.disclaimer}
            </div>`;
          } else {
            html += `<div class="healthy-message">
              <p>${data.message}</p>
            </div>`;
          }

          html += `</div>`;

          resultDiv.innerHTML = html;
          submitBtn.disabled = false;
          submitBtn.innerHTML = '<span class="btn-text">🔍 Analyze Another Image</span>';
        })
        .catch(err => {
          resultDiv.innerHTML = `<div class="error">❌ Network Error: ${err.message}</div>`;
          submitBtn.disabled = false;
          submitBtn.innerHTML = '<span class="btn-text">🔍 Analyze Image</span>';
        });
    }
  </script>

</body>
</html>

In [ ]:
%%writefile templates/welcome.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>Welcome - DR Detection</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>
<body>
  <div class="welcome-wrapper">
    <div class="welcome-container">
      <div class="welcome-icon">👁️</div>
      <h1 class="welcome-title">Diabetic Retinopathy Detection System</h1>
      <p class="welcome-subtitle">
        AI-Powered Early Detection for Vision Protection
      </p>

      <div class="feature-grid">
        <div class="feature-item">
          <div class="feature-icon">🔬</div>
          <h3>Advanced CNN Analysis</h3>
          <p>Deep learning model trained on thousands of retinal images</p>
        </div>

        <div class="feature-item">
          <div class="feature-icon">🤖</div>
          <h3>Personalized Recommendations</h3>
          <p>AI-generated care steps tailored to your specific condition</p>
        </div>

        <div class="feature-item">
          <div class="feature-icon">⚡</div>
          <h3>Instant Results</h3>
          <p>Get detailed analysis in seconds with confidence scoring</p>
        </div>

        <div class="feature-item">
          <div class="feature-icon">🎯</div>
          <h3>Severity Classification</h3>
          <p>Multi-dimensional assessment of retinal damage patterns</p>
        </div>
      </div>

      <div class="info-box">
        <h3>📌 About This Tool</h3>
        <p>
          This system uses state-of-the-art deep learning to analyze fundus photographs
          for signs of diabetic retinopathy. Our CNN model extracts visual features and
          classifies severity levels, while an AI assistant generates personalized care
          recommendations based on detected patterns.
        </p>
        <p class="info-warning">
          ⚠️ <strong>Important:</strong> This is a screening tool, not a diagnostic device.
          Always consult qualified ophthalmologists for medical diagnosis and treatment.
        </p>
      </div>

      <button onclick="location.href='/upload'" class="welcome-btn">
        Begin Analysis →
      </button>

      <div class="footer-note">
        <p>Powered by TensorFlow, Hugging Face Transformers & Mistral-7B</p>
      </div>
    </div>
  </div>
</body>
</html>

In [ ]:
%%writefile static/style.css
* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}

body {
  font-family: 'Segoe UI', system-ui, -apple-system, sans-serif;
  background: linear-gradient(135deg, #0a1628 0%, #1a2f4d 50%, #0d1b2a 100%);
  color: #e8f1f5;
  min-height: 100vh;
  line-height: 1.6;
}

/* ========== HEADER ========== */
.header {
  text-align: center;
  padding: 2rem 1rem 1rem;
  background: rgba(255, 255, 255, 0.03);
  border-bottom: 2px solid rgba(90, 139, 214, 0.3);
  margin-bottom: 2rem;
}

.header h1 {
  font-size: 2.2rem;
  margin-bottom: 0.5rem;
  background: linear-gradient(135deg, #5a8bd6, #89c4f4);
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  background-clip: text;
}

.tagline {
  font-size: 1rem;
  color: #a8c5e0;
  font-weight: 300;
}

/* ========== CONTAINER ========== */
.container {
  max-width: 900px;
  margin: 0 auto;
  padding: 0 1.5rem 3rem;
}

/* ========== UPLOAD SECTION ========== */
.upload-section {
  background: rgba(255, 255, 255, 0.05);
  backdrop-filter: blur(10px);
  border: 1px solid rgba(90, 139, 214, 0.3);
  border-radius: 16px;
  padding: 2.5rem;
  margin-bottom: 2rem;
  box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
}

.upload-section h2 {
  font-size: 1.8rem;
  margin-bottom: 0.5rem;
  color: #89c4f4;
}

.instruction {
  color: #b8d4ea;
  margin-bottom: 1.5rem;
  font-size: 0.95rem;
}

/* ========== FILE INPUT ========== */
.file-input-wrapper {
  position: relative;
  margin-bottom: 1.5rem;
}

input[type='file'] {
  display: none;
}

.file-label {
  display: flex;
  align-items: center;
  justify-content: space-between;
  width: 100%;
  padding: 1.2rem 1.5rem;
  background: linear-gradient(135deg, #1e3a5f, #2d4a6f);
  border: 2px dashed #5a8bd6;
  border-radius: 12px;
  cursor: pointer;
  transition: all 0.3s ease;
  color: #c6d9ea;
  font-size: 1rem;
}

.file-label:hover {
  background: linear-gradient(135deg, #2d4a6f, #3a5a7f);
  border-color: #89c4f4;
  transform: translateY(-2px);
}

.upload-icon {
  font-size: 1.5rem;
}

/* ========== IMAGE PREVIEW ========== */
.image-preview {
  display: none;
  margin: 1.5rem 0;
  text-align: center;
}

.image-preview img {
  max-width: 100%;
  max-height: 300px;
  border-radius: 12px;
  border: 2px solid #5a8bd6;
  box-shadow: 0 4px 16px rgba(0, 0, 0, 0.4);
}

/* ========== SUBMIT BUTTON ========== */
.submit-btn {
  width: 100%;
  padding: 1rem 2rem;
  background: linear-gradient(135deg, #0066cc, #004c99);
  color: white;
  border: none;
  border-radius: 10px;
  font-size: 1.1rem;
  font-weight: 600;
  cursor: pointer;
  transition: all 0.3s ease;
  box-shadow: 0 4px 15px rgba(0, 102, 204, 0.4);
}

.submit-btn:hover:not(:disabled) {
  background: linear-gradient(135deg, #0052a3, #003d7a);
  transform: translateY(-2px);
  box-shadow: 0 6px 20px rgba(0, 102, 204, 0.6);
}

.submit-btn:disabled {
  opacity: 0.6;
  cursor: not-allowed;
}

/* ========== RESULTS SECTION ========== */
.results-section {
  background: rgba(255, 255, 255, 0.05);
  backdrop-filter: blur(10px);
  border: 1px solid rgba(90, 139, 214, 0.3);
  border-radius: 16px;
  padding: 2rem;
  box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
  animation: fadeIn 0.5s ease;
}

.results-section.hidden {
  display: none;
}

.result-header h2 {
  color: #89c4f4;
  margin-bottom: 1.5rem;
  font-size: 1.8rem;
}

.result-card {
  background: rgba(255, 255, 255, 0.08);
  border-radius: 12px;
  padding: 1.5rem;
}

/* ========== PREDICTION DISPLAY ========== */
.prediction-main {
  margin-bottom: 2rem;
}

.prediction-main h3 {
  color: #89c4f4;
  margin-bottom: 1rem;
  font-size: 1.3rem;
}

.prediction-text {
  font-size: 1.2rem;
  color: #e8f1f5;
  margin-bottom: 1.5rem;
  line-height: 1.8;
}

/* ========== CONFIDENCE BAR ========== */
.confidence-bar {
  margin-top: 1rem;
}

.confidence-label {
  font-size: 0.9rem;
  color: #b8d4ea;
  margin-bottom: 0.5rem;
}

.progress-bar {
  width: 100%;
  height: 28px;
  background: rgba(0, 0, 0, 0.3);
  border-radius: 14px;
  overflow: hidden;
  border: 1px solid rgba(90, 139, 214, 0.4);
}

.progress-fill {
  height: 100%;
  background: linear-gradient(90deg, #0066cc, #00aaff);
  border-radius: 14px;
  transition: width 1s ease;
  display: flex;
  align-items: center;
  justify-content: flex-end;
  padding-right: 10px;
}

.confidence-value {
  text-align: right;
  margin-top: 0.5rem;
  font-weight: 600;
  color: #89c4f4;
  font-size: 1.1rem;
}

/* ========== SEVERITY INFO ========== */
.severity-info {
  background: rgba(255, 255, 255, 0.05);
  border-left: 4px solid #ff9800;
  padding: 1.2rem;
  border-radius: 8px;
  margin-bottom: 2rem;
}

.severity-badge {
  display: inline-block;
  padding: 0.5rem 1rem;
  border-radius: 20px;
  font-weight: 600;
  margin-bottom: 0.8rem;
  font-size: 0.95rem;
}

.severity-early-mild {
  background: rgba(255, 193, 7, 0.2);
  color: #ffd54f;
  border: 1px solid #ffc107;
}

.severity-moderate {
  background: rgba(255, 152, 0, 0.2);
  color: #ffb74d;
  border: 1px solid #ff9800;
}

.severity-moderately-severe {
  background: rgba(255, 87, 34, 0.2);
  color: #ff8a65;
  border: 1px solid #ff5722;
}

.severity-severe-advanced {
  background: rgba(244, 67, 54, 0.2);
  color: #ef5350;
  border: 1px solid #f44336;
}

.severity-desc {
  color: #d0e3f0;
  margin: 0.5rem 0;
}

.affected-area {
  color: #b8d4ea;
  font-size: 0.9rem;
}

.affected-area strong {
  color: #89c4f4;
}

/* ========== RECOVERY STEPS ========== */
.recovery-section {
  margin-top: 2rem;
  border-top: 2px solid rgba(90, 139, 214, 0.3);
  padding-top: 2rem;
}

.recovery-section h3 {
  color: #89c4f4;
  margin-bottom: 1rem;
  font-size: 1.3rem;
}

.steps-container {
  background: rgba(255, 255, 255, 0.08);
  border-radius: 10px;
  padding: 1.5rem;
  border: 1px solid rgba(90, 139, 214, 0.2);
}

.steps-text {
  font-family: 'Segoe UI', system-ui, sans-serif;
  white-space: pre-wrap;
  color: #e8f1f5;
  font-size: 0.95rem;
  line-height: 1.8;
  margin: 0;
}

/* ========== DISCLAIMER ========== */
.disclaimer {
  margin-top: 1.5rem;
  padding: 1rem;
  background: rgba(244, 67, 54, 0.1);
  border: 1px solid rgba(244, 67, 54, 0.3);
  border-radius: 8px;
  color: #ffcdd2;
  font-size: 0.9rem;
  font-weight: 500;
}

.healthy-message {
  background: rgba(76, 175, 80, 0.1);
  border: 1px solid rgba(76, 175, 80, 0.3);
  border-radius: 8px;
  padding: 1.5rem;
  margin-top: 1rem;
  color: #c8e6c9;
}

/* ========== LOADING ========== */
.loading {
  text-align: center;
  padding: 3rem 1rem;
}

.spinner {
  border: 4px solid rgba(255, 255, 255, 0.1);
  border-top: 4px solid #5a8bd6;
  border-radius: 50%;
  width: 50px;
  height: 50px;
  animation: spin 1s linear infinite;
  margin: 0 auto 1rem;
}

@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}

.loading p {
  color: #b8d4ea;
  margin: 0.5rem 0;
}

.loading-sub {
  font-size: 0.9rem;
  color: #8ba9c4;
}

/* ========== ERROR ========== */
.error {
  background: rgba(244, 67, 54, 0.15);
  border: 1px solid #f44336;
  border-radius: 8px;
  padding: 1.5rem;
  color: #ffcdd2;
  text-align: center;
}

/* ========== WELCOME PAGE ========== */
.welcome-wrapper {
  min-height: 100vh;
  display: flex;
  align-items: center;
  justify-content: center;
  padding: 2rem 1rem;
}

.welcome-container {
  max-width: 1000px;
  text-align: center;
  animation: fadeIn 0.8s ease;
}

.welcome-icon {
  font-size: 5rem;
  margin-bottom: 1rem;
  animation: float 3s ease-in-out infinite;
}

.welcome-title {
  font-size: 2.8rem;
  margin-bottom: 1rem;
  background: linear-gradient(135deg, #5a8bd6, #89c4f4, #b8d4ea);
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  background-clip: text;
}

.welcome-subtitle {
  font-size: 1.3rem;
  color: #b8d4ea;
  margin-bottom: 3rem;
  font-weight: 300;
}

/* ========== FEATURE GRID ========== */
.feature-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(220px, 1fr));
  gap: 1.5rem;
  margin: 3rem 0;
}

.feature-item {
  background: rgba(255, 255, 255, 0.05);
  backdrop-filter: blur(10px);
  border: 1px solid rgba(90, 139, 214, 0.3);
  border-radius: 12px;
  padding: 1.8rem;
  transition: all 0.3s ease;
}

.feature-item:hover {
  transform: translateY(-5px);
  background: rgba(255, 255, 255, 0.08);
  border-color: #5a8bd6;
  box-shadow: 0 8px 24px rgba(90, 139, 214, 0.2);
}

.feature-icon {
  font-size: 2.5rem;
  margin-bottom: 1rem;
}

.feature-item h3 {
  color: #89c4f4;
  font-size: 1.1rem;
  margin-bottom: 0.8rem;
}

.feature-item p {
  color: #b8d4ea;
  font-size: 0.9rem;
  line-height: 1.6;
}

/* ========== INFO BOX ========== */
.info-box {
  background: rgba(255, 255, 255, 0.05);
  border: 1px solid rgba(90, 139, 214, 0.3);
  border-radius: 12px;
  padding: 2rem;
  margin: 3rem 0;
  text-align: left;
}

.info-box h3 {
  color: #89c4f4;
  margin-bottom: 1rem;
  font-size: 1.3rem;
}

.info-box p {
  color: #d0e3f0;
  margin-bottom: 1rem;
  line-height: 1.8;
}

.info-warning {
  background: rgba(255, 152, 0, 0.1);
  border-left: 3px solid #ff9800;
  padding: 1rem;
  border-radius: 6px;
  color: #ffcc80;
}

/* ========== WELCOME BUTTON ========== */
.welcome-btn {
  padding: 1.2rem 3rem;
  background: linear-gradient(135deg, #0066cc, #004c99);
  color: white;
  border: none;
  border-radius: 12px;
  font-size: 1.2rem;
  font-weight: 600;
  cursor: pointer;
  margin-top: 2rem;
  transition: all 0.3s ease;
  box-shadow: 0 6px 20px rgba(0, 102, 204, 0.4);
}

.welcome-btn:hover {
  background: linear-gradient(135deg, #0052a3, #003d7a);
  transform: translateY(-3px);
  box-shadow: 0 8px 28px rgba(0, 102, 204, 0.6);
}

.footer-note {
  margin-top: 3rem;
  padding-top: 2rem;
  border-top: 1px solid rgba(90, 139, 214, 0.2);
}

.footer-note p {
  color: #8ba9c4;
  font-size: 0.9rem;
}

/* ========== ANIMATIONS ========== */
@keyframes fadeIn {
  from {
    opacity: 0;
    transform: translateY(20px);
  }
  to {
    opacity: 1;
    transform: translateY(0);
  }
}

@keyframes float {
  0%, 100% {
    transform: translateY(0);
  }
  50% {
    transform: translateY(-20px);
  }
}

/* ========== RESPONSIVE ========== */
@media (max-width: 768px) {
  .header h1 {
    font-size: 1.8rem;
  }

  .welcome-title {
    font-size: 2rem;
  }

  .welcome-subtitle {
    font-size: 1rem;
  }

  .feature-grid {
    grid-template-columns: 1fr;
  }

  .upload-section {
    padding: 1.5rem;
  }
}

📘 Kill Previous Processes

This ensures Flask and ngrok do not conflict:

- Stops earlier Flask sessions  
- Stops older ngrok tunnels  
- Prevents "port already in use" errors  

Safe to run every time before starting server.


In [ ]:
# Kill any previous processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

📘  Checking Port 8000 (User Instructions)

If server fails, port 8000 may be occupied.

Run:
!lsof -i :8000

If you see:
python   12345 LISTEN

Kill it with:
!kill -9 12345

Then launch Flask again.


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 4209 2548


📘  Run Flask App in Background

Starts backend without blocking the notebook:

!nohup python app.py > flask.log 2>&1 &

Logs are stored in flask.log


In [ ]:
# Run Flask in the background
!nohup python app.py > flask.log 2>&1 &


📘  Ngrok Setup

Ngrok provides a public HTTPS link.

Your ngrok token was removed for safety.

To use ngrok:
1. Get token → https://dashboard.ngrok.com/get-started/your-authtoken  
2. Add inside notebook:

conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

3. Start tunnel:

public_url = ngrok.connect(8000)

Shareable app link appears here.


In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# Optional: check logs
!sleep 3 && tail -n 20 flask.log


In [ ]:
!tail -n 50 flask.log

In [ ]:
# 🎉 Deployment Successful!

Your AI-powered Diabetic Retinopathy Detection System is now live.

This web app includes:
✔ CNN vision model
✔ Severity analysis
✔ Agentic AI recovery plan generator
✔ Beautiful interactive UI

Share your public ngrok URL to allow others to test the app.
